In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import math
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('sonar_dataset.csv', header=None)

In [ ]:
x = df.drop(60, axis='columns')
y = df[60]

In [ ]:
y = pd.get_dummies(y, drop_first=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

In [ ]:
class TrainDataset(Dataset):
    def __init__(self):
        self.x = torch.tensor(x_train.values, dtype=torch.float32)
        self.y = torch.tensor(y_train.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [ ]:
traindataset = TrainDataset()
dataloader_train = DataLoader(traindataset,
                       batch_size=8,
                       shuffle=True,
                       num_workers=0,
                       drop_last=False,
                       pin_memory=True)

In [ ]:
class Rock_Classifier(torch.nn.Module):
    def __init__(self):
        super(Rock_Classifier, self).__init__()
        self.layer1 = torch.nn.Linear(60, 30)
        self.layer2 = torch.nn.Linear(30, 15)
        self.layer3 = torch.nn.Linear(15, 1)
        self.dropout = torch.nn.Dropout(0.2, inplace=True)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        out = self.relu(self.dropout(self.layer1(x)))
        out = self.relu(self.dropout(self.layer2(out)))
        out = self.sigmoid(self.dropout(self.layer3(out)))
        return out

In [ ]:
rock_classifier = Rock_Classifier()

In [ ]:
optimizer = torch.optim.Adam(rock_classifier.parameters(), lr=0.008, betas=(0.9, 0.99))#, weight_decay=0.001)

In [ ]:
loss_function = torch.nn.BCELoss()

In [ ]:
# l2_lambda = 0.01
for epoch in range(200):
    total_loss = 0.0
    for idx, (x1, y1) in enumerate(dataloader_train):
        optimizer.zero_grad()
        pred = rock_classifier(x1)
#         l2_norm = sum(abs(p).sum() for p in rock_classifier.parameters())
        lossval = loss_function(pred, y1)
#         lossval = lossval + l2_lambda * l2_norm
        lossval.backward(retain_graph=True)
        optimizer.step()
        
        total_loss += lossval.item()
    average_loss = total_loss / traindataset.__len__()
    if epoch % 5 == 0:
        print("Average loss: {}".format(average_loss))

In [ ]:
class TestDataset(Dataset):
    def __init__(self):
        self.x = torch.tensor(x_test.values, dtype=torch.float32)
        self.y = torch.tensor(y_test.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [ ]:
testdataset = TestDataset()
dataloader_test = DataLoader(testdataset,
                       batch_size=1,
                       shuffle=True,
                       num_workers=0,
                       drop_last=False,
                       pin_memory=True)

In [ ]:
rock_classifier = rock_classifier.cpu()
rock_classifier.eval()

y_test_true = []
y_test_pred = []
for idx, (x1, y1) in enumerate(dataloader_test):
    y_test_true.append(y1.numpy()[0][0])
    pred = rock_classifier(x1)
    pred_val = np.round(pred.detach().numpy()[0][0])
    y_test_pred.append(pred_val)

In [ ]:
dataloader_train_2 = DataLoader(traindataset, batch_size=1, shuffle=True, num_workers=0, drop_last=False, pin_memory=True)

y_train_true = []
y_train_pred = []
for idx, (x1, y1) in enumerate(dataloader_train_2):
    y_train_true.append(y1.numpy()[0][0])
    pred = rock_classifier(x1)
    pred_val = np.round(pred.detach().numpy()[0][0])
    y_train_pred.append(pred_val)

In [ ]:
print(classification_report(y_train_true, y_train_pred))

In [ ]:
print(classification_report(y_test_true, y_test_pred))